In [6]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import f1_score

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

batch_size = 64
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

trainset = torchvision.datasets.CIFAR10(root='./data', 
                                        train=True,
                                        download=True, 
                                        transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, 
                                          batch_size=batch_size,
                                          shuffle=True, 
                                          num_workers=4)

Files already downloaded and verified


In [6]:
# 모델 정의
model = torchvision.models.squeezenet1_1(pretrained=True)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
model.to(device)

# 손실 함수 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-3)

Downloading: "https://download.pytorch.org/models/squeezenet1_1-b8a52dc0.pth" to C:\Users\Suhyeon/.cache\torch\hub\checkpoints\squeezenet1_1-b8a52dc0.pth
100.0%


cuda:0


In [7]:
# 4. 학습용 데이터를 사용하여 신경망 학습

epochs = 50

for epoch in range(epochs):

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # [inputs, labels]의 목록인 data로부터 입력을 받은 후;
        # cuda(gpu)로 데이터를 넘겨줘야 한다.
        inputs, labels = data[0].to(device), data[1].to(device)

        # 변화도(Gradient) 매개변수를 0으로 만들고
        optimizer.zero_grad()

        # 순전파 + 역전파 + 최적화를 한 후
        outputs = model(inputs)
        # 손실 함수의 loss 결과 값을 그래프로 보면 될 것
        # batch마다 출력하고 있으니 epoch 끝날 때 마다 평균내서 출력해야 한다.
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

#     # EarlyStopping
#     es(running_loss / i)
    print(f'[{epoch + 1}] loss: {running_loss / i:.3f}')
#     # Early Stop Check
#     if es.early_stop:
#         break

    # F1-score 구할 때 function은 cpu에서 돌아가야 할 것
    # 다시 cpu로 가져오기 위해서 data.to('cpu')

print('Finished Training')


# 학습한 모델 저장
PATH = './cifar_squeeze_net.pth'
torch.save(model.state_dict(), PATH)

[1] loss: 2.602
[2] loss: 2.198
[3] loss: 2.117
[4] loss: 1.943
[5] loss: 1.581
[6] loss: 1.243
[7] loss: 1.061
[8] loss: 0.940
[9] loss: 0.857
[10] loss: 0.783
[11] loss: 0.731
[12] loss: 0.688
[13] loss: 0.660
[14] loss: 0.619
[15] loss: 0.589
[16] loss: 0.572
[17] loss: 0.545
[18] loss: 0.513
[19] loss: 0.490
[20] loss: 0.479
[21] loss: 0.453
[22] loss: 0.449
[23] loss: 0.422
[24] loss: 0.420
[25] loss: 0.395
[26] loss: 0.399
[27] loss: 0.380
[28] loss: 0.365
[29] loss: 0.350
[30] loss: 0.339
[31] loss: 0.339
[32] loss: 0.335
[33] loss: 0.309
[34] loss: 0.312
[35] loss: 0.326
[36] loss: 0.294
[37] loss: 0.292
[38] loss: 0.282
[39] loss: 0.291
[40] loss: 0.286
[41] loss: 0.271
[42] loss: 0.262
[43] loss: 0.265
[44] loss: 0.242
[45] loss: 0.258
[46] loss: 0.232
[47] loss: 0.263
[48] loss: 0.239
[49] loss: 0.230
[50] loss: 0.234
Finished Training


In [9]:
# 저장했던 모델 불러오기
PATH = './cifar_squeeze_net.pth'
model = torchvision.models.squeezenet1_1(pretrained = False)
model.load_state_dict(torch.load(PATH))
# outputs = model(images)

<All keys matched successfully>

In [20]:
# 모델 저장 후 시험용 데이터로 신경망 검사할 때 

transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

batch_size = 64

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [13]:
# 5. 모델 저장 후 시험용 데이터로 신경망 검사

testset = torchvision.datasets.CIFAR10(root='./data', 
                                       train=False,
                                       download=True, 
                                       transform=transform)

testloader = torch.utils.data.DataLoader(testset, 
                                         batch_size=batch_size, 
                                         shuffle=False, 
                                         num_workers=4)

dataiter = iter(testloader)
images, labels = next(dataiter)

Files already downloaded and verified


In [18]:
# 시험용 데이터 불러오기

dataiter = iter(testloader)
images, labels = next(dataiter)

model = torchvision.models.squeezenet1_1(pretrained = True)
model.load_state_dict(torch.load(PATH))
model.eval()

# f1-score 측정(no Gradients)
with torch.no_grad():
    y_true = []
    y_pred = []
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        y_true.extend(labels.numpy())
        y_pred.extend(predictions.cpu().numpy())

f1 = f1_score(y_true, y_pred, average='macro')
print(f'F1 score: {f1:.3f}')

F1 score: 0.756


In [21]:
# outputs = model(images)

# 가장 높은 값을 갖는 인덱스(index)를 뽑기
_, predicted = torch.max(outputs, 1)
print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}'
                              for j in range(4)))

# 각 분류(class)에 대한 예측값 계산을 위해 준비
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# 변화도는 여전히 필요하지 않습니다
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        # 각 분류별로 올바른 예측 수를 모읍니다
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# 각 분류별 정확도(accuracy)를 출력합니다
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Predicted:  horse cat   ship  plane
Accuracy for class: plane is 75.6 %
Accuracy for class: car   is 83.6 %
Accuracy for class: bird  is 67.8 %
Accuracy for class: cat   is 63.9 %
Accuracy for class: deer  is 69.8 %
Accuracy for class: dog   is 63.5 %
Accuracy for class: frog  is 84.2 %
Accuracy for class: horse is 80.1 %
Accuracy for class: ship  is 82.3 %
Accuracy for class: truck is 82.5 %
